# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [79]:
# В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

In [80]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import normaltest

In [81]:
df_petersburg = pd.DataFrame({'petersburg': petersburg})
df_magadan = pd.DataFrame({'magadan': magadan})
data = pd.concat([df_petersburg, df_magadan], axis=1)
len(petersburg)==len(magadan)   # Равны ли длины списков?

False

In [82]:
# Длины списков не равны, в общем датафрейме будут NaN
f_df = data.copy()  # копия исходного датафрейма
# Поскольку информации о наличии выбросов нет, заменим NaN медианным значением
f_df = f_df.fillna(f_df.median()) # f_df - датафрейм без NaN, data - исходный

## Проверка данных на нормальность

In [83]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
alpha = 0.05   # Установим уровень значимости

### Тест Шапиро-Уилка

In [84]:
def get_shapiro(df, col):
    """Аргументы: df - датафрейм, col - столбец анализируемого датафрейма;
    возвращаемые значения - значение "p" и интерпретация гипотез "H" """
    global alpha
    _, p = shapiro(df[col])
    # Интерпретация 
    if p > alpha:
	    H = H0
    else:
	    H = Ha
    return p, H

for col in list(f_df.columns):
    print(f'{col}:  p={round(get_shapiro(f_df, col)[0], 3)}   {get_shapiro(f_df, col)[1]}')

petersburg:  p=0.106   Данные распределены нормально
magadan:  p=0.036   Данные не распределены нормально (мы отвергаем H0)


Одна группа не прошла тест на нормальность, корректнее будет проводить непараметрический корреляционный тест.

### Тест Д’Агостино

In [85]:
def get_n_t(df, i):
    """Аргументы: df - датафрейм, i - номер столбца анализируемого датафрейма;
    возвращаемые значения - значение "p" и интерпретация гипотез "H" """    
    global alpha
    _, p = normaltest(df)
    # normaltest возвращает двустороннюю вероятность для проверки гипотезы
    # Интерпретация 
    if p[i] > alpha/2:
    	H = H0
    else:
    	H = Ha
    return p[i], H 

for i in range(f_df.shape[1]):
    print(
    f'{list(f_df.columns)[i]}:  p={round(get_n_t(f_df, i)[0],3)}   {get_n_t(f_df, i)[1]}')

petersburg:  p=0.035   Данные распределены нормально
magadan:  p=0.154   Данные распределены нормально


c:\Users\Rui\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Тест Д’Агостино  вычисляет эксцесс и перекос по данным, чтобы определить, отклоняется ли распределение данных от нормального распределения. Перекос (асимметрию skewness) по исходным данным определить невозможно.


"skewtest is not valid with less than 8 samples; 7 samples were given"

В столбце 'petersburg' до замены Nan на медианное значение 7 значений. 

Вообще тест Д’Агостино использовать некорректно для числа наблюдений меньше 20. 

Имеем две количественных переменных, причем одна группа не прошла проверку на нормальность по тесту Шапиро-Уилка.
 
Поэтому выбираем тест на корреляцию Спирмена.

## Корреляция Спирмана

In [86]:
from scipy.stats import spearmanr
corr, p = spearmanr(f_df['petersburg'], f_df['magadan'])
print(f'corr = {round(corr, 6)}, p = {round(p, 6)}')
f_df.corr(method = 'spearman')

corr = 0.155691, p = 0.712762


,petersburg,magadan
petersburg,1.000000,0.155691
magadan,0.155691,1.000000


In [87]:
if p>alpha:
    print(f"{round(p, 3)} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.")
else:
    print(f"{round(p, 3)} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутствии зависимости между переменными.")

0.713 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.


Делаем вывод, что зависимость между размерами мидий из Магадана и Санкт-Петербурга отсутствует (очень слабая, corr меньше 0.3).

## Независимый T-тест 

Независимый T-тест проверяет разницу между одной и той же переменной из разных совокупностей и используется на выборках размером меньше 30. Зависимая переменная (размер мидии) является количественной. Группы происходят из разных совокупностей. Но для T-теста необходимо, чтобы данные выборок имели распределение близкое к нормальному. Его применение в данном случае не совсем корректно.

In [88]:
from scipy.stats import ttest_ind
H0 = 'Нет значимой разницы между размерами мидий из Магадана и Санкт-Петербурга.'
Ha = 'Есть значимая разница между размерами мидий из Магадана и Санкт-Петербурга.'

In [89]:
def t_test(data):
    print("*** Результаты независимого T-теста ***")
    test_results = ttest_ind(f_df['petersburg'], f_df['magadan'], equal_var=True)

    p = round(test_results[1],3)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(f_df)

*** Результаты независимого T-теста ***
0.003 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий из Магадана и Санкт-Петербурга.


## Критерий Уилкоксона
Критерий Уилкоксона - это непараметрическая версия парного Т-теста. Его применение в данном случае более корректно.

In [90]:
from scipy.stats import wilcoxon

test_results = wilcoxon(f_df['petersburg'], f_df['magadan'])

p = round(test_results[1],3)
print("*** Результаты по тесту Уилкоксона ***")
if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

*** Результаты по тесту Уилкоксона ***
0.023 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий из Магадана и Санкт-Петербурга.


Оба теста показали наличие значимой разницы между размерами мидий из разных городов.

Вычислим разницу между средними размерами мидий из Магадана и Санкт-Петербурга.

In [91]:
# Разница между средними размерами мидий
diff = abs(data['petersburg'].mean()-data['magadan'].mean()) 
diff_perc = round(diff/(max(data['magadan'].mean(), data['petersburg'].mean()))*100, 2)

print(f"Относительная разница между средними размерами мидий из Магадана и Санкт-Петербурга {diff_perc} %")

Относительная разница между средними размерами мидий из Магадана и Санкт-Петербурга 24.58 %
